In [1]:
import json
import re
import time
import string
import pandas as pd
import matplotlib.pyplot as plt
from decouple import Config, RepositoryEnv
from ibm_watsonx_ai import Credentials, APIClient
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters

In [2]:
# Load the environment variables using python-decouple
# The .env file should be in the root of the project
# The .env file should NOT be committed to the repository

config = Config(RepositoryEnv(".env.paul"))

# Load the credentials
WX_API_KEY = config("WX_API_KEY")

In [3]:
credentials = Credentials(
    url = "https://us-south.ml.cloud.ibm.com",
    api_key = WX_API_KEY
)
client = APIClient(
    credentials=credentials, 
    project_id="68126b74-155e-4a70-aa2c-1781dfad87f6"
)

In [4]:
# --- Setup the Model ---
params = TextGenParameters(
    temperature=0.05,           # Light creativity
    max_new_tokens=300          # Enough for descriptive text
)

model = ModelInference(
    api_client=client,
    params=params,
    model_id="mistralai/mistral-large",
)

In [5]:
def create_prompt(sign_name: str) -> str:
    return (
        f"You are an American Sign Language (ASL) teacher.\n\n"
        f"Please clearly explain how to perform the ASL sign on a beginner level for the letter '{sign_name}'. "
        f"Use simple language and full sentences. Do not assume any prior knowledge about ASL.\n\n"
        f"Here you can find 5 examples:\n"
        f"1. sign: 'Hello' - explaination: Begin with the side of your index finger against your forehead and then move your hand up and away from your head.\n"
        f"2. sign: 'customer' - explaination: Begin with your hands on each side of the top of your chest with your palms oriented toward each other and your thumbs touching your chest. Move your hands of your chest and bring them down and press them against your midsection.\n"
        f"3. sign: 'become' - explaination: Begin with both palms oriented towards each other with your hands perpendicular to each other. Then, rotate your wrist until your hands are perpendicular to each other in the opposite direction.\n"
        f"4. sign: 'certain' - explaination: Begin with your index finger touching your mouth and pointing up. Then, bring it forward and down until your index finger is facing forwards.\n"
        f"5. sign: 'all' - explaination: Begin with both hands in front of you. Your non-dominant hand should be closer to you and be oriented towards yourself. Your dominant hand should be oriented away from yourself. Rotate your dominant hand so that its palm is oriented toward yourself and then rest the back of your dominant hand against the palm of your non-dominant hand\n\n"
        f"Only output the explanation once. Do not include any other text. If appropriate, use less tokens than available.\n\n"
    )


In [6]:
# --- Generate ASL Description ---
def generate_asl_description(sign_name: str) -> str:
    prompt = create_prompt(sign_name)
    
    response = model.generate(prompt=prompt)
    generated_text = response['results'][0]['generated_text']
    
    return generated_text

In [7]:
# --- Generate all 26 ASL letter descriptions and record time ---
start = time.perf_counter()

descriptions = {
    letter: generate_asl_description(letter).strip()
    for letter in string.ascii_uppercase
}

elapsed = time.perf_counter() - start
print(f"Finished in {elapsed:.2f} s")


Finished in 40.46 s
